<a href="https://colab.research.google.com/github/ggaoshen/graph_wavenet/blob/sg-final/CS224W_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CS224W - Final Project**

Description goes here.

# Device
You might need to use GPU for this Colab.

Please click `Runtime` and then `Change runtime type`. Then set the `hardware accelerator` to **GPU**.

# Installation

In [36]:
# Install torch geometric
import os
import torch
if 'IS_GRADESCOPE_ENV' not in os.environ:
  torch_version = str(torch.__version__)
  scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  !pip3 install torch-scatter -f $scatter_src
  !pip3 install torch-sparse -f $sparse_src
  !pip3 install torch-geometric
  !pip3 install -q git+https://github.com/snap-stanford/deepsnap.git
  !pip3 install -U -q PyDrive

Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1.html

[notice] A new release of pip is available: 23.0 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1.html

[notice] A new release of pip is available: 23.0 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip


In [37]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  !nvcc --version
  !python -c "import torch; print(torch.version.cuda)"

zsh:1: command not found: nvcc


None


In [38]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  import torch
  print(torch.__version__)
  import torch_geometric
  print(torch_geometric.__version__)

1.13.1
2.4.0


In [39]:
# Import GraphWaveNet module
# !git clone -b sg-experiment https://github.com/ggaoshen/graph_wavenet.git # NOTE: choose the right branch
# %load graph_wavenet/src/graphwavenet.py
import sys
sys.path.append('graph_wavenet/src/')
from graphwavenet import GraphWaveNet
import util as util
from util import masked_rmse, masked_mse, masked_mae, masked_mape, metric, temporal_dataset_split

In [40]:
!pip3 install -U torch-geometric-temporal


[notice] A new release of pip is available: 23.0 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip


In [41]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context



# Temporal Datasets
# from torch_geometric_temporal.dataset import METRLADatasetLoader
# loader = METRLADatasetLoader()
# dataset = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=12)
# iterator = iter(dataset)
# print("METRLADatasetLoader", next(iterator))
# print("METRLADatasetLoader", next(iterator))


from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
dataset = ChickenpoxDatasetLoader().get_dataset(lags=8) # consistent with chickenpox paper
iterator = iter(dataset)
print("ChickenpoxDatasetLoader", next(iterator))
print("ChickenpoxDatasetLoader", next(iterator))


# from torch_geometric_temporal.dataset import WikiMathsDatasetLoader
# dataset = WikiMathsDatasetLoader().get_dataset(lags=14)
# iterator = iter(dataset)
# print("WikiMathsDatasetLoader", next(iterator))
# print("WikiMathsDatasetLoader", next(iterator))


# from torch_geometric_temporal.dataset import TwitterTennisDatasetLoader
# dataset = TwitterTennisDatasetLoader().get_dataset()
# print("TwitterTennisDatasetLoader", next(iter(dataset)))


# Graph summaries
# METRLADatasetLoader Data(x=[207, 2, 12], edge_index=[2, 1722], edge_attr=[1722], y=[207, 12])
# ChickenpoxDatasetLoader Data(x=[20, 4], edge_index=[2, 102], edge_attr=[102], y=[20])
# WikiMathsDatasetLoader Data(x=[1068, 14], edge_index=[2, 27079], edge_attr=[27079], y=[1068])

ChickenpoxDatasetLoader Data(x=[20, 8], edge_index=[2, 102], edge_attr=[102], y=[20])
ChickenpoxDatasetLoader Data(x=[20, 8], edge_index=[2, 102], edge_attr=[102], y=[20])


In [42]:
# Run Chickenpox Dataset

import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd

# training
from tqdm import tqdm
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache() # clear cuda cache

from torch_geometric_temporal.signal import temporal_signal_split
train_ratio = 0.8
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=train_ratio)
offset = int(dataset.snapshot_count * train_ratio) # starting index for test set

in_dim = dataset[0].num_node_features # 8 treat lagged inputs as node features
out_dim = 1
num_nodes = dataset[0].num_nodes # 1068
timesteps_to_predict = 10 # 10, 20, 40 week forecast horizon
epochs = 200
lrate = 0.0001
wdecay = 0.001
save_path = "store/checkpoint"

util.extensions_enabled = True
print("Graph Wavenet Extension for Performance Improvements: {}".format(util.extensions_enabled))

model = GraphWaveNet(
    num_nodes=num_nodes,
    in_channels=in_dim,
    out_channels=out_dim,
    out_timesteps=timesteps_to_predict,
).to(device)

if not os.path.exists(save_path):
    os.makedirs(save_path)

print("start training...", flush=True)
his_loss = []
val_time = []
train_time = []
best_epoch = 0

optimizer = torch.optim.Adam(model.parameters(), lr=lrate, weight_decay=wdecay)

scheduler = None
clip = None

if util.extensions_enabled:
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.99)
    clip = 5
    epochs = 50

model.train()

def prepare_n_period_y(dataset):

    res = []
    for data in dataset:
        res.append(data.y)
    res = torch.stack(res, dim=0)

    return res

y_all = prepare_n_period_y(dataset)
training_curve_dict = {"epoch_train_loss": []}

for epoch in tqdm(range(epochs)):
    train_loss = []
    t1 = time.time()

    for i, snapshot in enumerate(train_dataset):
        x_train = snapshot.x.reshape(-1, num_nodes, in_dim).to(device)
        if timesteps_to_predict == 1:
            y_train = snapshot.y.to(device)
        else:
            y_train = y_all[i : i + timesteps_to_predict,:].to(device)

        pred = model(x_train, snapshot.edge_index, snapshot.edge_attr).squeeze()
        loss = masked_mse(pred, y_train, 0.0) # mean squared error for loss
        loss.backward()

        if util.extensions_enabled:
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        if util.extensions_enabled:
            scheduler.step()

        optimizer.zero_grad()

        train_loss.append(loss.item())

    mtrainloss = np.mean(train_loss)
    training_curve_dict['epoch_train_loss'].append(mtrainloss)
    print(f"training loss: {mtrainloss}")


# eval
model.eval()
loss = 0
for i, snapshot in enumerate(test_dataset):
    if i + timesteps_to_predict > test_dataset.snapshot_count:
        break

    x_test = snapshot.x.reshape(-1, num_nodes, in_dim).to(device)

    if timesteps_to_predict == 1:
        y_test = snapshot.y.to(device)
    else:
        y_test = y_all[offset+i : offset+i + timesteps_to_predict,:].to(device)


    with torch.no_grad():
        pred = model(x_test, snapshot.edge_index, snapshot.edge_attr).squeeze()
    loss += masked_mse(pred, y_test, 0.0) # mean squared error as loss

loss = loss / (i+1)
loss = loss.item()
print("Test MSE Loss: {:.4f}".format(loss))


# store training loss
pd.DataFrame(training_curve_dict).to_csv("training_curve.csv")

from google.colab import files
files.download('training_curve.csv')


Graph Wavenet Extension for Performance Improvements: True
start training...


  2%|▏         | 1/50 [00:10<08:40, 10.62s/it]

training loss: 0.9676242742823755


  4%|▍         | 2/50 [00:21<08:24, 10.51s/it]

training loss: 0.951149949022546


...

training loss: 0.9367790733532207


100%|██████████| 50/50 [08:37<00:00, 10.35s/it]

training loss: 0.936582082569054


Test MSE Loss: 0.9576


ModuleNotFoundError: No module named 'google.colab'

In [43]:
training_curve_dict["epoch_train_loss"]

[0.9676242742823755,
 0.951149949022546,
 0.9455509232357144,
 0.9425896940162269,
 0.9402643811230252,
 0.9388957972479303,
 0.93831085133389,
 0.9376601587117809,
 0.9368611871969046,
 0.9371511423824037,
 0.9369071703404188,
 0.9370966077350625,
 0.9366364542394876,
 0.9365718980615095,
 0.9369772766376051,
 0.9370812701561102,
 0.9366132067561876,
 0.9366890484012845,
 0.9366053721844787,
 0.9367367275286375,
 0.9365811903983718,
 0.9369450423048764,
 0.9364961250117276,
 0.9366765397169241,
 0.9363589960945452,
 0.9366046300475918,
 0.9365479999380868,
 0.9365652676989756,
 0.9364063007757067,
 0.9370151242422985,
 0.9362905974613457,
 0.9366968588267521,
 0.9366122781685213,
 0.9367014459220738,
 0.9365284113486002,
 0.9368718633519076,
 0.9365718769454738,
 0.936560070269355,
 0.9366208900220511,
 0.9362758073138028,
 0.9367317246364021,
 0.9364329920354776,
 0.9365164942267101,
 0.9370631711512077,
 0.9366091117626283,
 0.9364306485870989,
 0.9365125967961986,
 0.93679233437812

In [ ]:
!nvidia-smi

In [ ]:
# Reset GPU RAM if GPU out of memory

# !pip install numba

# from numba import cuda

# cuda.select_device(0) # choosing second GPU

# cuda.close()